In [3]:
import numpy as np
import numpy.core.defchararray as npc
from random import randint
import matplotlib.pyplot as plt
!pip install pandas
import pandas as pd



  Using cached pandas-1.3.4-cp39-cp39-macosx_10_9_x86_64.whl (11.6 MB)


In [4]:

data_makanan = np.array([
    [["00", "Apel", 52], ["01", "Pisang", 89], [
        "10", "Mangga", 60], ["11", "Semangka", 30]],

    [["00", "Roti", 265], ["01", "Kentang", 87], [
        "10", "Jagung", 86], ["11", "Ketela", 159]],
        
    [["00", "Ikan", 206], ["01", "Daging Sapi", 143],
        ["10", "Ayam", 239], ["11", "Telur", 155]]
])

df = pd.DataFrame(data_makanan[0])
df

,0,1,2
0,00,Apel,52
1,01,Pisang,89
2,10,Mangga,60
3,11,Semangka,30


In [6]:

"""
Membuat satu kombinasi makanan, satu kombinasi terdiri dari satu buah, satu karbohidrat dan
satu lauk. kombinasi bersifat random.
kombinasi satu kali makan mewakili satu kromosom
"""


def menu_sekali_makan():
    kromosom = np.array([])
    for menu in data_makanan:
        random_index = randint(0, 3)
        kromosom = np.append(kromosom, menu[random_index, 0])
    return kromosom

menu_sekali_makan()

array(['11', '01', '10'], dtype='<U32')

In [7]:
"""
Satu individu diwujudkan dalam menu seminggu. menu seminggu inilah yang akan dihitung fitness 
nya terhadap kriteria fitness.
"""

def menu_seminggu():
    individu = np.array([])
    for hari in range(7):
        individu = np.append(individu, menu_sekali_makan())
    individu = individu.reshape(7, 3)
    return individu

menu_seminggu()

array([['00', '00', '11'],
       ['10', '00', '00'],
       ['11', '10', '00'],
       ['10', '01', '10'],
       ['11', '00', '01'],
       ['00', '11', '11'],
       ['10', '01', '10']], dtype='<U32')

In [54]:
"""
Membangkitkan generasi pertama
"""

def buat_leluhur(jumlah_populasi):
    leluhur = np.array([])
    for i in range(jumlah_populasi):
        leluhur = np.append(leluhur, menu_seminggu())
    leluhur = leluhur.reshape(jumlah_populasi, 7, 3)
    # print(leluhur)
    return leluhur

generasi1 = buat_leluhur(3)

In [55]:
"""
Cek fitness untuk setiap generasi yang dilahirkan.
cek fitness I adalah menghitung total kalori untuk setiap kali makan. jika > 500 kalori maka
individu tersebut dinyatakan kurang fit. jika kurang dari 500 maka diberikan skor fitnes 10
"""


def dibawah_kalori_maksimal(menu_sekali_makan, maks_kalori):
    # cari kalori buah
    buah, cols = np.where(data_makanan[0] == menu_sekali_makan[0])
    print(buah)
    kalori_buah = int(data_makanan[0][buah, 2][0])

    # cari kalori karbo
    karbo, cols = np.where(data_makanan[1] == menu_sekali_makan[1])
    kalori_karbo = int(data_makanan[1][karbo, 2][0])

    # cari kalori lauk
    lauk, cols = np.where(data_makanan[2] == menu_sekali_makan[2])
    kalori_lauk = int(data_makanan[2][lauk, 2][0])

    # jumlahkan total kalori
    total_kalori = kalori_buah + kalori_karbo + kalori_lauk
    # print(total_kalori)
    # cek apakah kalori melebihi batas maksimal
    if total_kalori < maks_kalori:
        return True
    else:
        return False

contoh = menu_seminggu()
print(contoh)
dibawah_kalori_maksimal(contoh, 400)

[['11' '11' '00']
 ['01' '01' '11']
 ['11' '00' '01']
 ['11' '00' '00']
 ['11' '01' '11']
 ['11' '00' '11']
 ['10' '01' '11']]
[3]


True

In [56]:
"""
Cek menu sebelumnya, skor diberikan jika menu tidak sama, minimal skor 0 maksimal 15.
"""

def cek_menu_sebelumnya(menu_sebelum, menu_sekarang):
    skor = 0
    if menu_sebelum[0] != menu_sekarang[0]:
        skor += 5
    if menu_sebelum[1] != menu_sekarang[1]:
        skor += 5
    if menu_sebelum[2] != menu_sekarang[2]:
        skor += 5
    return skor

cek_menu_sebelumnya(contoh[2], contoh[3])

5

In [57]:
"""
cek fitness setiap individu dalam satu generasi
"""


def cek_fitness(generasi, maks_kalori):
    fit_status_individu = np.array([], dtype='int16')
    nomor_individu = 0
    best_score = 0
    worst_score = 1000
    best_individu = np.array([])
    worst_individu = np.array([])
    for individu in generasi:
        fit_status_individu = np.append(fit_status_individu, nomor_individu)
        nomor_individu += 1
        skor = 0
        kalorifit = 60
        menu_sebelum = None
        for kromosom in individu:
            if dibawah_kalori_maksimal(kromosom, maks_kalori) == False:
                kalorifit = 0
            if menu_sebelum is not None:
                skor = skor + cek_menu_sebelumnya(menu_sebelum, kromosom)
            menu_sebelum = kromosom
        fit_status_individu = np.append(fit_status_individu, kalorifit)
        fit_status_individu = np.append(fit_status_individu, skor)
        fit_status_individu = np.append(fit_status_individu, kalorifit + skor)
        if (kalorifit + skor) > best_score:
            best_score = kalorifit + skor
            best_individu = individu
        if worst_score > (kalorifit + skor):
            worst_score = kalorifit + skor
            worst_individu = individu
    fit_status_individu = fit_status_individu.reshape(nomor_individu, 4)
    # print(fit_status_individu)
    fit_stat = fit_status_individu[:, [3]]
    # cari yang terbaik berdasarkan skor terbaik
    # get best score
    # best = np.amax(fit_stat)
    # worst= np.amin(fit_stat)
    # print(fit_stat)
    # print("nilai individu terbaik = " + str(np.amax(fit_stat)) + " Nilai individu terburuk = "+ str(np.amin(fit_stat)))
    return fit_status_individu, best_score, worst_score, best_individu, worst_individu

_ = cek_fitness(generasi1, 400)

[3]
[1]
[2]
[0]
[3]
[1]
[3]
[2]
[1]
[2]
[3]
[2]
[0]
[1]
[0]
[1]
[0]
[3]
[1]
[1]
[0]


In [42]:
"""
membuat persentase kemungkinan berpasangan dengan roulete.
"""

def make_roulete(fit_status_individu):
    sum_fit_status_individu = fit_status_individu.sum(axis=0)
    new_fit_status_individu = np.array([], dtype='int16')
    angka = 0
    for individu in fit_status_individu:
        angka = angka + \
            int(round(individu[3]/sum_fit_status_individu[3]*100, 0))
        individu = np.append(individu, angka)
        new_fit_status_individu = np.append(new_fit_status_individu, individu)
    new_fit_status_individu = new_fit_status_individu.reshape(
        len(fit_status_individu), 5)
    # print(new_fit_status_individu)
    return new_fit_status_individu

In [44]:
# cari yang beruntung untuk kawin dan carikan pasangannya

def do_roulete(fit_status_individu, jumlah_populasi):
    # untuk buat generasi dengan ukuran populasi sama dibutuhkan pasangan sejumlah 1/2 jumlah populasi
    jumlah_pasangan = int(jumlah_populasi/2)
    # untuk mencari jumlah persentase
    max_fit = np.amax(fit_status_individu[:, [4]])
    # print(max_fit)
    pasangan = np.array([], dtype='int16')
    # print(fit_status_individu)
    # membuat pasangan sejumlah yang ditetapkan
    individu_sebelumnya = None
    for pop in range(jumlah_populasi):
        random_index = randint(1, max_fit)  # nomer keberuntungan
        # ambil satu individu yang beruntung
        for individu in fit_status_individu:
            if random_index <= individu[4]:
                individu_sebelumnya = individu[0]
                pasangan = np.append(pasangan, individu[0])
                break
    # print(fit_status_individu)
    # print(pasangan)
    # reshape sesuai panjang array dibagi 2
    pasangan = pasangan.reshape(int(len(pasangan)/2), 2)
    return pasangan

In [47]:
def mutasi(individu):
    for i in range(2):
        random_index = randint(0, 6)
        ori = individu[random_index]
        mutan = npc.replace(ori, "0", "2")
        mutan = npc.replace(mutan, "1", "0")
        mutan = npc.replace(mutan, "2", "1")
        individu[random_index] = mutan
    return individu

In [48]:
def kawin(generasi, pasangan, probabilitas_kawin, probabilitas_mutasi):
    keturunan = np.array([])
    # print(pasangan)
    for couple in pasangan:
        # print(generasi[couple[1]])
        parent1 = generasi[couple[0]]
        parent2 = generasi[couple[1]]
        child1 = parent1
        child2 = parent2
        # print("=====")
        # print(child2)
        random_chance = randint(0, 100)
        # proses kawin
        if random_chance > (100 - probabilitas_kawin):
            for i in range(3):
                random_index = randint(0, 6)
                child1[random_index] = parent2[random_index]
            for i in range(3):
                random_index = randint(0, 6)
                child2[random_index] = parent1[random_index]
                # print("=====")
                # print(child2[random_index])

        # proses probabilitas mutasi generasi baru
        # print(child2)
        # print("=====")
        random_mutate = randint(0, probabilitas_mutasi)
        random_match = randint(0, probabilitas_mutasi)
        if random_mutate == random_match:
            child2 = mutasi(child2)
            child1 = mutasi(child1)

        keturunan = np.append(keturunan, child1)
        keturunan = np.append(keturunan, child2)
    # print(keturunan)
    keturunan = keturunan.reshape(-1, 7, 3)
    # print(generasi1)
    return keturunan

In [38]:
def printhasil(best, worst):
    hari = 1
    print("Susunan Menu terbaik : ")
    for menu in best:
        idxbuah, cols = np.where(data_makanan[0] == menu)
        nama_buah = data_makanan[0][idxbuah, 1][0]
        idxkarbo, cols = np.where(data_makanan[1] == menu)
        nama_karbo = data_makanan[1][idxkarbo, 1][0]
        idxlauk, cols = np.where(data_makanan[2] == menu)
        nama_lauk = data_makanan[2][idxlauk, 1][0]
        print("Hari ke-"+str(hari)+", Buah :"+nama_buah +
              ", makanan : "+nama_karbo+", Lauk: "+nama_lauk)
        hari += 1
    print("========================================================================")
    print("")
    print("Susunan Menu Terburuk : ")
    hari = 1
    for menu in worst:
        idxbuah, cols = np.where(data_makanan[0] == menu)
        nama_buah = data_makanan[0][idxbuah, 1][0]
        idxkarbo, cols = np.where(data_makanan[1] == menu)
        nama_karbo = data_makanan[1][idxkarbo, 1][0]
        idxlauk, cols = np.where(data_makanan[2] == menu)
        nama_lauk = data_makanan[2][idxlauk, 1][0]
        print("Hari ke-"+str(hari)+", Buah :"+nama_buah +
              ", makanan : "+nama_karbo+", Lauk: "+nama_lauk)
        hari += 1
    return hari

In [45]:

def eksekusi(jml_populasi, jml_iterasi, maks_kalori, probabilitas_kawin, probabilitas_mutasi):
    # print(type(maks_kalori))
    bw_individu = np.array([], dtype='int16')
    it = int(0)
    generasi1 = buat_leluhur(jml_populasi)
    fit_status = cek_fitness(generasi1, maks_kalori)
    best_score_of_all = fit_status[1]
    worst_score_of_all = fit_status[2]
    best_of_all = fit_status[3]
    worst_of_all = fit_status[4]
    bw_individu = np.append(bw_individu, it)
    bw_individu = np.append(bw_individu, int(fit_status[1]))
    bw_individu = np.append(bw_individu, int(fit_status[2]))
    new_fit_status = (make_roulete(fit_status[0]))
    pasangan = do_roulete(new_fit_status, jml_populasi)
    keturunan = kawin(generasi1, pasangan,
                      probabilitas_kawin, probabilitas_mutasi)
    for it in range(jml_iterasi):
        fit_status = cek_fitness(keturunan, maks_kalori)
        if fit_status[1] > best_score_of_all:
            best_score_of_all = fit_status[1]
            best_of_all = fit_status[3]
        if worst_score_of_all > fit_status[2]:
            worst_score_of_all = fit_status[2]
            worst_of_all = fit_status[4]
        bw_individu = np.append(bw_individu, it+1)
        bw_individu = np.append(bw_individu, int(fit_status[1]))
        bw_individu = np.append(bw_individu, int(fit_status[2]))
        new_fit_status = (make_roulete(fit_status[0]))
        pasangan = do_roulete(new_fit_status, jml_populasi)
        keturunan = kawin(keturunan, pasangan,
                          probabilitas_kawin, probabilitas_mutasi)
    bw_individu = bw_individu.reshape(jml_iterasi+1, 3)
    # print(bw_individu)
    return bw_individu, best_of_all, worst_of_all

In [66]:
populasi = 10
iterasi = 100 # Generasi
hasil = eksekusi(populasi, iterasi, 400, 70, 100)
bw_individu = hasil[0]
generation = bw_individu[:, [0]]
best = bw_individu[:, [1]]
worst = bw_individu[:, [2]]
generation = generation.reshape(-1)
best = best.reshape(-1)
worst = worst.reshape(-1)

[0]
[3]
[1]
[0]
[1]
[2]
[1]
[3]
[2]
[1]
[0]
[3]
[0]
[0]
[0]
[3]
[3]
[1]
[2]
[3]
[2]
[2]
[2]
[2]
[2]
[1]
[1]
[1]
[1]
[0]
[0]
[1]
[3]
[0]
[2]
[0]
[2]
[2]
[1]
[3]
[2]
[1]
[3]
[0]
[3]
[1]
[0]
[2]
[0]
[1]
[3]
[1]
[3]
[2]
[0]
[1]
[3]
[1]
[2]
[3]
[0]
[2]
[2]
[0]
[1]
[0]
[3]
[0]
[0]
[3]
[1]
[2]
[1]
[1]
[3]
[0]
[0]
[1]
[2]
[0]
[1]
[3]
[0]
[2]
[3]
[0]
[3]
[1]
[0]
[2]
[0]
[3]
[1]
[2]
[3]
[0]
[2]
[2]
[1]
[2]
[2]
[1]
[3]
[0]
[1]
[1]
[2]
[2]
[1]
[3]
[0]
[1]
[1]
[2]
[1]
[3]
[3]
[0]
[1]
[1]
[3]
[1]
[3]
[2]
[0]
[1]
[1]
[2]
[2]
[1]
[3]
[0]
[1]
[3]
[0]
[3]
[1]
[0]
[2]
[0]
[1]
[2]
[2]
[1]
[3]
[0]
[1]
[1]
[2]
[2]
[1]
[3]
[0]
[1]
[1]
[2]
[0]
[1]
[3]
[0]
[2]
[1]
[2]
[1]
[1]
[3]
[0]
[0]
[3]
[2]
[3]
[3]
[3]
[0]
[1]
[3]
[2]
[3]
[1]
[0]
[2]
[0]
[1]
[2]
[2]
[1]
[3]
[0]
[1]
[1]
[2]
[2]
[1]
[3]
[0]
[1]
[3]
[2]
[3]
[3]
[3]
[0]
[1]
[3]
[0]
[3]
[1]
[0]
[2]
[0]
[1]
[2]
[2]
[1]
[3]
[0]
[1]
[1]
[2]
[1]
[1]
[3]
[0]
[1]
[1]
[2]
[2]
[1]
[3]
[0]
[1]
[3]
[2]
[3]
[1]
[0]
[2]
[0]
[1]
[2]
[2]
[1]
[3]
[0]
[1]
[1]
[2]
[2]
[1]
[3]


In [67]:
generation

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100])

In [68]:
best

array([75, 85, 85, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80,
       80, 80, 85, 85, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 85,
       85, 85, 85, 85, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80,
       80, 80, 80, 80, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75,
       75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75,
       75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75])

In [69]:
worst

array([55, 70, 65, 65, 70, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80,
       80, 70, 70, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 80, 65, 65,
       80, 80, 80, 80, 80, 80, 80, 80, 70, 70, 70, 70, 70, 70, 70, 70, 70,
       70, 65, 60, 60, 65, 65, 65, 60, 60, 60, 60, 75, 75, 75, 75, 75, 75,
       75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75,
       75, 70, 70, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75])